In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pandas as pd
import numpy as np
import datetime
import os
import itertools
import pywt

#### Space-time prediction idea: Construct the first order adjacency matrix of the road segment to be predicted, and obtain the decomposed time part by singular value decomposition; The network structure of cnn+lstm+attention was used for prediction

### Get the data of each coil

Select the coils of nodes 4697-4806(5690) for prediction. The coils required for the construction of the first order adjacency matrix are: 1.4706-4697(5684) coils; 2.4651-4697(6135); 3. 4694-4697(6132) coil. The four coils form a spatial adjacency matrix

In [ ]:
data_5690=pd.read_csv('train_data/data/5690.csv')
data_5684=pd.read_csv('train_data/data/5684.csv')
data_6135=pd.read_csv('train_data/data/6135.csv')
data_6132=pd.read_csv('train_data/data/6132.csv')

In [ ]:
def findTime(x):
    return x.time()
def findDate(x):
    return x.date()

data_5690['FROMTIME'] = pd.to_datetime(data_5690['FROMTIME'])
data_5684['FROMTIME'] = pd.to_datetime(data_5684['FROMTIME'])
data_6135['FROMTIME'] = pd.to_datetime(data_6135['FROMTIME'])
data_6132['FROMTIME'] = pd.to_datetime(data_6132['FROMTIME'])

data_5690.index=data_5690['FROMTIME']
data_5684.index=data_5684['FROMTIME']
data_6135.index=data_6135['FROMTIME']
data_6132.index=data_6132['FROMTIME']

data_5690['FROMTIME-Time'] = data_5690['FROMTIME'].apply(findTime)  
data_5690['FROMTIME-Date'] = data_5690['FROMTIME'].apply(findDate)  

data_5684['FROMTIME-Time'] = data_5684['FROMTIME'].apply(findTime) 
data_5684['FROMTIME-Date'] = data_5684['FROMTIME'].apply(findDate) 

data_6135['FROMTIME-Time'] = data_6135['FROMTIME'].apply(findTime) 
data_6135['FROMTIME-Date'] = data_6135['FROMTIME'].apply(findDate)

data_6132['FROMTIME-Time'] = data_6132['FROMTIME'].apply(findTime)  
data_6132['FROMTIME-Date'] = data_6132['FROMTIME'].apply(findDate) 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM,Bidirectional
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras import regularizers
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10

In [ ]:
# read data
xianquan_5690 = data_5690.resample(datetime.timedelta(seconds=5 * 60)).sum()
df_5690= xianquan_5690.copy()

df_5690['FROMTIME'] = pd.to_datetime(df_5690.index)
df_5690['FROMTIME-Time'] = df_5690["FROMTIME"].apply(findTime)  # 找到时间
df_5690['FROMTIME-Date'] = df_5690["FROMTIME"].apply(findDate)  # 找到日期
df_5690=df_5690[(pd.to_datetime(df_5690["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(df_5690["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]

In [ ]:
xianquan_5684 = data_5684.resample(datetime.timedelta(seconds=5 * 60)).sum()
df_5684= xianquan_5684.copy()

df_5684['FROMTIME'] = pd.to_datetime(df_5684.index)
df_5684['FROMTIME-Time'] = df_5684["FROMTIME"].apply(findTime)  # 找到时间
df_5684['FROMTIME-Date'] = df_5684["FROMTIME"].apply(findDate)  # 找到日期
df_5684=df_5684[(pd.to_datetime(df_5684["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(df_5684["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]

In [ ]:
xianquan_6135 = data_6135.resample(datetime.timedelta(seconds=5 * 60)).sum()
df_6135= xianquan_6135.copy()

df_6135['FROMTIME'] = pd.to_datetime(df_6135.index)
df_6135['FROMTIME-Time'] = df_6135["FROMTIME"].apply(findTime)  # 找到时间
df_6135['FROMTIME-Date'] = df_6135["FROMTIME"].apply(findDate)  # 找到日期
df_6135=df_6135[(pd.to_datetime(df_6135["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(df_6135["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]

In [ ]:
xianquan_6132 = data_6132.resample(datetime.timedelta(seconds=5 * 60)).sum()
df_6132= xianquan_6132.copy()

df_6132['FROMTIME'] = pd.to_datetime(df_6132.index)
df_6132['FROMTIME-Time'] = df_6132["FROMTIME"].apply(findTime)  # 找到时间
df_6132['FROMTIME-Date'] = df_6132["FROMTIME"].apply(findDate)  # 找到日期
df_6132=df_6132[(pd.to_datetime(df_6132["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(df_6132["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]

### Prove spatial correlation

In [ ]:
co_6132=df_6132[(pd.to_datetime(df_6132['FROMTIME-Date'],format = '%Y-%m-%d')>= pd.to_datetime('2019-07-08',format = '%Y-%m-%d'))&(pd.to_datetime(df_6132["FROMTIME-Date"],format = '%Y-%m-%d')<= pd.to_datetime('2019-07-14',format = '%Y-%m-%d'))]
co_5690=df_5690[(pd.to_datetime(df_5690['FROMTIME-Date'],format = '%Y-%m-%d')>= pd.to_datetime('2019-07-08',format = '%Y-%m-%d'))&(pd.to_datetime(df_5690["FROMTIME-Date"],format = '%Y-%m-%d')<= pd.to_datetime('2019-07-14',format = '%Y-%m-%d'))]
co_5684=df_5684[(pd.to_datetime(df_5684['FROMTIME-Date'],format = '%Y-%m-%d')>= pd.to_datetime('2019-07-08',format = '%Y-%m-%d'))&(pd.to_datetime(df_5684["FROMTIME-Date"],format = '%Y-%m-%d')<= pd.to_datetime('2019-07-14',format = '%Y-%m-%d'))]
co_6135=df_6135[(pd.to_datetime(df_6135['FROMTIME-Date'],format = '%Y-%m-%d')>= pd.to_datetime('2019-07-08',format = '%Y-%m-%d'))&(pd.to_datetime(df_6135["FROMTIME-Date"],format = '%Y-%m-%d')<= pd.to_datetime('2019-07-14',format = '%Y-%m-%d'))]

In [ ]:
data_6132=np.array(co_6132["COUNT"])
data_5690=np.array(co_5690["COUNT"])
data_5684=np.array(co_5684["COUNT"])
data_6135=np.array(co_6135["COUNT"])

In [ ]:
fig=plt.figure(figsize= (20,5))
ax = fig.add_subplot(111)
d = '2019-07-08 08:00'
x = pd.date_range(d, periods=168, freq='5min')

ax.plot(data_6132,label='4694-4697')
ax.plot(data_5690,label='4697-4806')
ax.plot(data_5684,label='4706-4697')
ax.plot(data_6135,label='4651-4697')
plt.legend()

spearman correlation coefficient calculation

In [ ]:
import pandas as pd
import numpy as np

x1=pd.Series(data_5684)
y1=pd.Series(data_6135)

n=x1.count()

x1.index=np.arange(n)
y1.index=np.arange(n)
  
d=(x1.sort_values().index-y1.sort_values().index)**2
dd=d.to_series().sum()

p=1-n*dd/(n*(n**2-1))

r=x1.corr(y1,method='spearman')
print(r,p)

The correlation coefficient between 6132 and 5690 is 0.438; The correlation coefficient between 6132 and 5684 is 0.537; The correlation coefficient between 6132 and 6135 is 0.782. The correlation coefficient between 5690 and 5684 was 0.685. The correlation coefficient between 5690 and 6135 was 0.512. The correlation coefficient between 5684 and 6135 is 0.472

The order of correlation with 5690 coil is: 5684>6135>6132

### Spatial Temporal Forecast

#### Data processing, establishing spatial-temporal matrix

In [ ]:
from tensorflow.contrib.learn.python.learn.datasets import base
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing

In [ ]:
def findTime(x):
    return x.time()
def findDate(x):
    return x.date()

In [ ]:
data_5690=pd.read_csv('train_data/data/5690.csv')
data_5684=pd.read_csv('train_data/data/5684.csv')
data_6135=pd.read_csv('train_data/data/6135.csv')
data_6132=pd.read_csv('train_data/data/6132.csv')
allframes=pd.DataFrame()
for road_id in [5684,5690,6135,6132]:
    data_road_id=pd.read_csv('train_data/data/%s.csv'%road_id)
    data_road_id['FROMTIME'] = pd.to_datetime(data_road_id['FROMTIME'])
    data_road_id.index = data_road_id['FROMTIME']
    road_data1= data_road_id.resample(datetime.timedelta(seconds=5 * 60)).sum()
    allframes=pd.concat([allframes,road_data1])

In [ ]:
allframes.to_csv('train_data/cnn_lstm/allframes.csv')

In [ ]:
# Group and standardize data by time series
def group_by_time():
    """
    :param key: group by time
    :param zscore: z score standardization
    :param grouped: The result of grouping and standardizing the data by time
    :param vehicles: Convert the data into matrix form
    """
    frame = frames
    frame['FROMTIME'] = pd.to_datetime(frame['FROMTIME'], format='%Y-%m-%d %H:%M:%S')
    values = frame.groupby('FROMTIME')['COUNT'].apply(list)
    vehicles = []
    
    for i in range(len(values)):
        count=0
        for j in range(len(values[i])):
            if values[i][j]<0.01:
                count+=1
        if count==4:
            pass
        else:
            vehicles.append(values[i])
    vehicles = np.asarray(vehicles)
    #vechicles = vehicles.reshape((196, 288*FILE_NUMS))
    #vechicles = np.array([np.reshape(x, (288, FILE_NUMS)) for x in vechicles])
    return vehicles

frames=pd.read_csv('train_data/cnn_lstm/allframes.csv')
frames['FROMTIME'] = pd.to_datetime(frames['FROMTIME'])
frames['FROMTIME-Time'] = frames['FROMTIME'].apply(findTime)  # 找到时间
frames['FROMTIME-Date'] = frames['FROMTIME'].apply(findDate)  # 找到日期
frames=frames[(pd.to_datetime(frames["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(frames["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]
vehicles = group_by_time()
vehicles = [i for i in vehicles if len(i)==4]


scaler = preprocessing.MinMaxScaler()
samples = scaler.fit_transform(vehicles)

In [ ]:
frames_tmp=frames[pd.to_datetime(frames["FROMTIME-Date"],format = '%Y-%m-%d')==pd.to_datetime('2019-07-01',format = '%Y-%m-%d')]

In [ ]:
frames_tmp.index=frames_tmp["FROMTIME"]

In [ ]:
plt.plot(frames_tmp["COUNT"])

In [ ]:
"""
split data
"""

class DataSet(object):
    def __init__(self, flow, labels):
        self._index_in_epoch = 0
        self._epochs_completed = 0
        self._flow = flow
        self._labels = labels
        self._num_examples = flow.shape[0]
        pass

    @property
    def flow(self):
        return self._flow

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, shuffle = True):
        """
        Return the next 'batch_size' examples from this dataset
        """
        start = self._index_in_epoch
        # Shuffle for the first epoch
        if start == 0 and self._epochs_completed == 0 and shuffle:
            idx = np.arange(self._num_examples)
            np.random.shuffle(idx)
            self._flow = self.flow[idx]
            self._labels = self.labels[idx]

        # Go to the next epoch
        if start + batch_size > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Get the rest examples in this epoch
            rest_num_examples = self._num_examples - start
            flow_rest_part = self._flow[start:self._num_examples]
            labels_rest_part = self._labels[start:self._num_examples]

            # Shuffle the data
            if shuffle:
                idx0 = numpy.arrange(self._num_examples)
                numpy.random.shuffle(idx0)
                self._flow = self.flow[idx0]
                self._labels = self.labels[idx0]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            flow_new_part = self._flow[start:end]
            labels_new_part = self._labels[start:end]
            return numpy.concatenate((flow_rest_part, flow_new_part), axis=0), numpy.concatenate((labels_rest_part, labels_new_part), axis=0)

        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._flow[start:end], self._labels[start:end]

def create_data_sets():
    sample = samples
    look_back = 12
    interval = 0
    flow, labels = [], []
    
    for i in range(len(sample)-look_back-interval):
        flow.append(sample[i:(i+look_back)])
        labels.append(sample[i+look_back+interval])
    return np.asarray(flow), np.asarray(labels)


def read_data_sets():
    flow, labels = create_data_sets()
    validation_size = 4157
    train_flow, test_flow, train_labels, test_labels = train_test_split(flow, 
                                                                        labels, 
                                                                        test_size = 0.2,
                                                                        random_state=0)

    validation_flow = train_flow[validation_size:]
    validation_labels = train_labels[validation_size:]
    train_flow = train_flow[:validation_size]
    train_labels = train_labels[:validation_size]

    train = DataSet(train_flow, train_labels)
    test = DataSet(test_flow, test_labels)
    validation = DataSet(validation_flow, validation_labels)
    return base.Datasets(train=train, validation=validation, test=test)

In [ ]:
flow,label=create_data_sets()

#### Model Training

In [ ]:
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, Dropout, Flatten, LSTM
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
import keras.backend as K

# import input_data

# Convolution
kernel_size = [2, 2, 4]

# Training
time_steps = 12
batch_size = 64
epochs = 100

# load model
print('loading data...')

pems = read_data_sets()
flow_train = pems.train.flow
flow_train = flow_train.reshape((flow_train.shape[0], time_steps, 4, 1))
labels_train = pems.train.labels
flow_test = pems.test.flow
flow_test = flow_test.reshape((flow_test.shape[0], time_steps, 4, 1))
labels_test = pems.test.labels
flow_validation = pems.validation.flow
flow_validation = flow_validation.reshape((flow_validation.shape[0], time_steps, 4, 1))
labels_validation = pems.validation.labels

# construct model 
print('build model...')
model = Sequential()

# the first convolution layer
model.add(TimeDistributed(Conv1D(40,
                 kernel_size[1],
                 strides=1,
                 padding='valid'), input_shape=[time_steps, 4, 1]))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))

model.add(TimeDistributed(Conv1D(40, kernel_size[1], padding='valid')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))

model.add(TimeDistributed(Conv1D(40, kernel_size[0], padding='valid')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation('relu')))

model.add(TimeDistributed(Flatten()))
model.add(Dense(4))
model.add(Dropout(0.25))

model.add(LSTM(64, return_sequences=True))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(LSTM(128))
model.add(Activation('tanh'))  
model.add(Dropout(0.25))
model.add(Dense(4))

model.summary()

# model.load_weights("Model/cnn_lstm_final.h5")

model.compile(loss='mean_squared_error', 
              optimizer='rmsprop',
              metrics=['mae', 'cosine'])

filepath = "train_data/model/cnn_lstm.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# train
print('Train...')

model.fit(flow_train, labels_train, 
#          validation_split=0.33,
          epochs=epochs, 
          batch_size=batch_size,
#           callbacks=callbacks_list,
          validation_data=(flow_validation, labels_validation))

score = model.evaluate(flow_test, labels_test, batch_size=batch_size)
print('Test score:', score)

In [ ]:
model.save('train_data/model/cnn_lstm.h5')

#### result comparison

In [ ]:
from matplotlib.font_manager import FontProperties
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as md
import matplotlib.ticker as mt
import pandas as pd
import datetime as dt
from keras.models import load_model
import math

def MAPE(y_true, y_pred):
    """Mean Absolute Percentage Error
    Calculate the mape.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    # Returns
        mape: Double, result data for train.
    """

    y = [x for x in y_true if x > 0]
    y_pred = [y_pred[i] for i in range(len(y_true)) if y_true[i] > 0]

    num = len(y_pred)
    sums = 0

    for i in range(num):
        tmp = abs(y[i] - y_pred[i]) / y[i]
        sums += tmp

    mape = sums * (100 / num)

    return mape
def eva_regress(y_true, y_pred):
    """Evaluation
    evaluate the predicted resul.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    """

    mape = MAPE(y_true, y_pred)
    vs = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print('explained_variance_score:%f' % vs)
    print('mape:%f%%' % mape)
    print('mae:%f' % mae)
    print('mse:%f' % mse)
    print('rmse:%f' % math.sqrt(mse))
    print('r2:%f' % r2)

In [ ]:
flow, labels = create_data_sets()

In [ ]:
time_steps = 12

# scaler = preprocessing.MinMaxScaler()
# samples = scaler.fit_transform(vehicles)


flow, labels = create_data_sets()
x_test = flow_validation

labels_test = labels_validation

cnn_lstm_final_model = load_model('train_data/model/cnn_lstm.h5')

y_test = scaler.inverse_transform(labels_test)


x_test= x_test.reshape((x_test.shape[0], time_steps, 4, 1))
pred = cnn_lstm_final_model.predict(x_test)
cnn_lstm_final_pred = scaler.inverse_transform(pred)

for i in range(4):
    plt.figure(figsize= (20,5))
    plt.plot(y_test[40:,i])
    plt.plot(cnn_lstm_final_pred[40:,i])
for i in range(4):
    eva_regress(y_test[40:,i], cnn_lstm_final_pred[40:,i])

In [ ]:
flow, labels = create_data_sets()

In [ ]:
flow, labels = create_data_sets()
x_test = flow[5290:]
labels_test = labels[5290:]

cnn_lstm_final_model = load_model('train_data/model/cnn_lstm.h5')

y_test = scaler.inverse_transform(labels_test)


x_test= x_test.reshape((x_test.shape[0], time_steps, 4, 1))
pred = cnn_lstm_final_model.predict(x_test)
cnn_lstm_final_pred = scaler.inverse_transform(pred)

for i in range(4):
    plt.figure(figsize= (20,5))
    plt.plot(y_test[:,i])
    plt.plot(cnn_lstm_final_pred[:,i])
for i in range(4):
    eva_regress(y_test[:,i], cnn_lstm_final_pred[:,i])